# Ajustes lineales, no lineales y bondad de ajuste

## Ajuste lineal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def funcion_lineal(t, m, q):
    
    y = m*t + q
    
    return y

data = np.loadtxt('datos_lineal.txt', skiprows=2)

y = data[:, 0]
err_y = data[:, 1]
x = data[:, 2]


popt, pcov = curve_fit(funcion_lineal, x, y)
perr = np.sqrt(np.diag(pcov))  # esta línea calcula los errores de los parámetros "perr"
                               # del ajuste a partir de la matriz de covarianza "pcov"

m_fit = popt[0]
q_fit = popt[1]

err_m = perr[0]
err_q = perr[1]

print('m =', m_fit, '+/-', err_m)
print('q =', q_fit, '+/-', err_q)

## Ajuste no lineal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

#defino la función de ajuste
def funcion_seno(t, A, w, phi, y0):    
    y = A * np.sin(w*t + phi) + y0
    return y


data = np.loadtxt('datos_nolineal.txt', skiprows=2)


y = data[:, 0]
err_y = data[:, 1]
x = data[:, 2]


A_i = 2.0

T_i = 20.0
f_i = 1/T_i
w_i = (2*np.pi)* f_i

phi_i = 0.0

y0_i = 0.5

init_guess = [A_i, w_i, phi_i, y0_i]

# cruve_fit es la función que hace el ajuste de la función modelo a los datos  (x, y)

popt, pcov = curve_fit(funcion_seno, x, y, p0=init_guess)
perr = np.sqrt(np.diag(pcov))  # esta línea calcula los errores de los parámetros "perr"
                               # del ajuste a partir de la matriz de covarianza "pcov"

# guardamos los resultados del ajuste

A_fit = popt[0]
w_fit = popt[1]
phi_fit = popt[2]
y0_fit = popt[3]

err_A = perr[0]
err_w = perr[1]
err_phi = perr[2]
err_y0 = perr[3]

# mostramos por pantalla los valores del ajuste

print('A =', A_fit, '+/-', err_A)
print('w =', w_fit, '+/-', err_ω)
print('phi =', phi_fit, '+/-', err_Φ)
print('y0 =', y0_fit, '+/-', err_y0)

# para graficar el modelo hacemos un array de abscisas (xx) de más puntos que los datos
# ajuste es el array de evaluar la funcion_seno sobre este array xx

xmin = 0
xmax = 60
xx = np.linspace(xmin, xmax, num=1000)
ajuste = funcion_seno(xx, A_fit, w_fit, phi_fit, y0_fit)

plt.plot(xx, ajuste, 'r-', linewidth=2.0, zorder=5)


## Goodness of fit

### Estimación de $\chi^2$

$$\chi^2 = \sum \left( \frac{y_{observed}-y_{model}}{\sigma} \right)^2 $$

El número de grados de libertad es 

$$\nu = n-1-dof$$

donde $n$ es el número de datos y $dof$ el número de grados de libertad en el ajuste. 

### Reduced $\chi^2$ 

$$red \chi = \chi^2 / \nu $$


### p-value 

Sale de 

$$ p = \int_{\chi^2}^{inf} f(u,\nu)du$$

donde $f$ es la distribución de probabilidad de $\chi^2$ con $\nu$ grados de libertad.



### $R^2$ 
$$R^2 = 1-\frac{SS_{Tot}}{SS_{res}}$$

In [ ]:
def chi_square(y_data, y_model, sigma):
    return np.sum((y_data/sigma - y_model/sigma)**2)

def r_chi_square(chi, y_data, dof):
    nu = len(y_data) - 1 - dof
    if nu < 1:
        raise ValueError('length of y_data - 1 - dof should be greater than 1')
    return chi / nu

def p_value(chi, y_data, dof):
    nu = len(y_data) - 1 - dof
    if nu < 1:
        raise ValueError('length of y_data - 1 - dof should be greater than 1')
    
    p = 1 - st.chi2.cdf(chi, nu)
    return p

def r_squared(y_data, y_model):
    ss_res = np.sum((y_data - y_model)**2)
    ss_tot = np.sum((y_data - np.mean(y_data))**2)
    return 1 - (ss_res / ss_tot)